In [11]:
from dotenv import dotenv_values
import os

envs = ["secret.env", "training.env"]

for fenv in envs:
    file = os.path.join("env", fenv)
    config = dotenv_values(file)  # load sensitive variables
    print(config.keys())
    for c, v in config.items():
        os.environ[c] = v

odict_keys(['WANDB_API_KEY'])
odict_keys(['WANDB_NAME', 'WANDB_NOTES', 'WANDB_NOTEBOOK_NAME'])


In [2]:
import wandb

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: Currently logged in as: wilber-quito (deepsat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch

torch.cuda.is_available()

True

In [4]:
import torch

torch.cuda.device_count()

1

In [5]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [6]:
import os

max(1, os.cpu_count() -1)

11

In [7]:
import os
import wandb
import torch
from lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from src.model import NeuralNetwork
from src.data import HousePricingDataModule
from src.util import fake_name

project_name = os.environ["WANDB_NAME"]

patience = 50
max_epochs = 500
accelerator = "gpu" if torch.cuda.is_available() else "cpu"

# Setting up the training configuration
config = {
    "accelerator": accelerator,
    "max_epochs": max_epochs,
    "patience": patience
}

test_results = dict()

for _, batch_size in enumerate([256, 512]):

    for i in range(6):

        run_name = fake_name()

        # Preparing data to be used
        data_module = HousePricingDataModule(batch_size=batch_size)
        data_module.prepare_data()

        # Requesting to data module the number of different features that are in the data
        in_features = data_module.in_features()

        config["batch_size"] = batch_size
        config["in_features"] = in_features

        wandb.init(
            job_type="training",
            name=run_name,
            project=project_name,
            config=config,
        )

        # Defining the model to be training
        model = NeuralNetwork(input_size=wandb.config["in_features"])

        checkpoint_name = f"{wandb.run.name}-{wandb.run.id}"

        # Defining the logger instance the lighning will use as default logging
        logger = WandbLogger(
            log_model="all",
            checkpoint_name=checkpoint_name
        )

        # Define how the model registry work
        checkpoint_callback = ModelCheckpoint(
            every_n_epochs=1,
            monitor="val_loss",
            mode = "min"
        )

        # Defining early stop configuration
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            mode="min",
            verbose=True,
            patience=wandb.config["patience"]
        )

        # Defines the training instance
        trainer = Trainer(
            accelerator=wandb.config["accelerator"],
            max_epochs=wandb.config["max_epochs"],
            logger=logger,
            log_every_n_steps=1,
            callbacks=[checkpoint_callback, early_stopping_callback]
        )

        trainer.fit(model, datamodule=data_module)

        test_result = trainer.test(model, datamodule=data_module)
        test_results[checkpoint_name] = test_result

        wandb.finish()

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[INFO]: Input size: 244
[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 215 K 
------------------------------------
215 K     Trainable params
0         Non-trainable params
215 K     Total params
0.864     Total estimated model params size (MB)


[INFO]: Setting up fit dataset/s
[INFO]: Train dataset size: 981
[INFO]: Validation dataset size: 173
Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Monitored metric val_loss did not improve in the last 50 records. Best score: 7024665088.000. Signaling Trainer to stop.


Epoch 51: 100%|██████████| 2/2 [00:00<00:00, 17.90it/s, v_num=qn2c]
[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Setting up test dataset/s
[INFO]: Test dataloader size: 3
Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 134.71it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss              24297048064.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
test_loss,▁
train_loss,█▁▄▇▇███▇█▇█▇▇███▇█▇█▇▇▇▇█▇▇▇▇▇▆▆▆▇▆▆▆▅▄
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▁▁▇█████████████████████████▇▇▇▇▇▇▆▆▆▆▆▅
epoch,52
test_loss,24297048064.0
train_loss,24385630208.0
trainer/global_step,104
val_loss,25643569152.0


In [8]:
test_results

{'Trevor_PTRZDD7M-wdeloj2e': [{'test_loss': 731680896.0}],
 'Scott_2PLKJYWM-0i4v9bts': [{'test_loss': 5276584448.0}],
 'Jessica_TI3MA5KG-gjeqqj22': [{'test_loss': 660979584.0}],
 'Nicole_YIL69KUL-17rqq9jc': [{'test_loss': 620553088.0}],
 'Leslie_M4EX6CT0-po6185wt': [{'test_loss': 656894912.0}],
 'Brian_RT18UQJB-3z7v3738': [{'test_loss': 654139392.0}],
 'Maria_B2X8HQTD-072cv3yj': [{'test_loss': 36679454720.0}],
 'Brian_F4MF8WSS-wwokesfs': [{'test_loss': 5263298560.0}],
 'Rebecca_P4QSR8JQ-4qi1nyd1': [{'test_loss': 5273787392.0}],
 'Daniel_ED290V7A-1wrinaj2': [{'test_loss': 5268858880.0}],
 'Tasha_X5RDUEUR-lum47bv0': [{'test_loss': 690038016.0}],
 'Tabitha_Y4OMZJ2U-mjutqn2c': [{'test_loss': 24297048064.0}]}

In [10]:
sss = {name: value[0]["test_loss"] for name, value in test_results.items()}
min_sss = min(sss, key=sss.get)
max_sss = max(sss, key=sss.get)
print(f"Best model in test dataset: {min_sss} - {test_results[min_sss]}")
print(f"Worst model in test dataset: {max_sss} - {test_results[max_sss]}")

Best model in test dataset: Nicole_YIL69KUL-17rqq9jc - [{'test_loss': 620553088.0}]
Worst model in test dataset: Maria_B2X8HQTD-072cv3yj - [{'test_loss': 36679454720.0}]
